In [302]:
import requests
import os
import tabula
import pandas as pd
from datetime import datetime, timedelta
import gspread
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials

In [303]:
def read_tables_from_url(url):
    response = requests.get(url)
    with open('temp.pdf', 'wb') as file:
        file.write(response.content)

    # Read tables from the PDF file
    tables = tabula.read_pdf('temp.pdf', pages='all', multiple_tables=True, stream=True, pandas_options={'header': None})

    # Remove the temporary PDF file
    os.remove('temp.pdf')

    return tables

In [304]:
# tables = read_tables_from_url("https://reporting.wrldc.in/dailyreports/PSP/2019/January/WRLDC_PSP_Report_31-01-2019.pdf")
# tables[1]

In [305]:
def preprocess_table(tables,date,states):
    date=date.strftime("%d-%m-%Y")

    # SOLVNG THE ISSUE OF DIFFERENT SPELLINGS OF CHHATTISGARH
    column_data = tables[1].iloc[:, 0].tolist()
    start_index = next((i for i, value in enumerate(column_data) if value in ["CHHATTISGARH", "CHHATISGARH"]), None)
    if start_index is not None:
        temp_table = tables[1].iloc[start_index:]
        temp_table = temp_table.reset_index(drop=True)
        temp_table[0][0] = "CHHATTISGARH"
    
    # SOLVING THE ISSUE OF DIFFERENT SPLITTNG OF MADHYA PRADESH IN DIFFERENT ROWS

    index = temp_table.index[temp_table[0] == "MADHYA"].values[0]
    temp_table.loc[index] = temp_table.loc[index+1]
    temp_table.drop([index+1,index+2],axis=0,inplace=True)
    temp_table = temp_table.reset_index(drop=True)
    temp_table[0][index]="MADHYA PRADESH"

    # EXPANDING THE COLUMNS READ AS SINGLE COLUMN

    temp_table.drop(columns=[3,4,5],axis=1,inplace=True)
    temp_table=temp_table.T.reset_index(drop=True).T
    temp_table[[2,3,4,5]]=temp_table[2].str.split(expand=True)

    # SOLVING PROBLEM OF DADAR AND NAGAR HAVELI AND DAMAN AND DIU COMBINED AS ONE IN RECENT YEARS

    if "DNHDDPDCL" not in temp_table.iloc[:, 0].values:
        temp_table.loc[1]=temp_table.loc[2]
        temp_table.drop([2,3],axis=0,inplace=True)
        temp_table = temp_table.reset_index(drop=True)
        for i in range(1,temp_table.shape[1]):
            if temp_table[i][1] == "-" or temp_table[i][2] == '-':
                temp_table[i][1]='-'
            else:
                temp_table[i][1]=float(temp_table[i][1])+float(temp_table[i][2])
        temp_table.drop(2,axis=0,inplace=True)
        temp_table=temp_table.reset_index(drop=True)
        temp_table[0][1]="DNHDDPDCL"

    # REMOVING UNNECESSARY ROWS

    index=[]
    index = [x for x, value in enumerate(temp_table.iloc[:, 0].tolist()) if value in ["AMNSIL", "ESIL", "Region"]]
    for i in index:
        temp_table.drop(i,axis=0,inplace=True)
    temp_table.reset_index(drop=True,inplace=True)

    temp_table.columns=['State','Thermal','Hydro','Wind','Solar','Others','Demand Met','Shortage']
    temp_table.insert(0,'Date',date)
    for index, row in temp_table.iterrows():
        state = row['State']
        if state not in states.keys():
            states[state] = pd.DataFrame(columns=temp_table.columns)
        states[state] = pd.concat([states[state], pd.DataFrame([row])], ignore_index=True)
    
    
    return states


In [306]:
# column_data = tables[1].iloc[:, 0].tolist()
# start_index = next((i for i, value in enumerate(column_data) if value in ["CHHATTISGARH", "CHHATISGARH"]), None)

# # Select the rows starting from the identified index
# if start_index is not None:
#     temp_table = tables[1].iloc[start_index:]
#     temp_table = temp_table.reset_index(drop=True)
#     temp_table[0][0] = "CHHATTISGARH"

# temp_table

In [307]:
# index = temp_table.index[temp_table[0] == "MADHYA"].values[0]
# temp_table.loc[index] = temp_table.loc[index+1]
# temp_table.drop([index+1,index+2],axis=0,inplace=True)
# temp_table = temp_table.reset_index(drop=True)
# temp_table[0][index]="MADHYA PRADESH"

In [308]:
# temp_table.drop(columns=[3,4,5],axis=1,inplace=True)
# temp_table=temp_table.T.reset_index(drop=True).T
# temp_table[[2,3,4,5]]=temp_table[2].str.split(expand=True)

In [309]:
# temp_table

In [310]:
# if "DNHDDPDCL" not in temp_table.iloc[:, 0].values:
#     temp_table.loc[1]=temp_table.loc[2]
#     temp_table.drop([2,3],axis=0,inplace=True)
#     temp_table = temp_table.reset_index(drop=True)
#     for i in range(1,temp_table.shape[1]):
#         if temp_table[i][1] == "-" or temp_table[i][2] == '-':
#             temp_table[i][1]='-'
#         else:
#             temp_table[i][1]=float(temp_table[i][1])+float(temp_table[i][2])
#     temp_table.drop(2,axis=0,inplace=True)
#     temp_table=temp_table.reset_index(drop=True)
#     temp_table[0][1]="DNHDDPDCL"


In [311]:
# index=[]
# index = [x for x, value in enumerate(temp_table.iloc[:, 0].tolist()) if value in ["AMNSIL", "ESIL", "Region"]]
# for i in index:
#     temp_table.drop(i,axis=0,inplace=True)
# temp_table.reset_index(drop=True,inplace=True)

In [312]:
months = ['January','February', 'March', 'April', 'May', 'June', 'July', 'August', 'September','October','November','December']
month_dict = {index + 1: month for index, month in enumerate(months)}

start_date = datetime(2019, 2, 19)  # Specify the start date
end_date = datetime(2019, 2, 21)  # Specify the end date

In [313]:
states={}
current_date = start_date
while current_date <= end_date:
    # https://reporting.wrldc.in/dailyreports/PSP/2023/May/WRLDC_PSP_Report_31-05-2023.pdf
    url=f'https://reporting.wrldc.in/dailyreports/PSP/{current_date.year}/{month_dict[current_date.month]}/WRLDC_PSP_Report_{current_date.strftime("%d-%m-%Y")}.pdf'
    tables=read_tables_from_url(url)
    preprocess_table(tables,current_date,states)
    current_date += timedelta(days=1)

Got stderr: Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 06, 2023 12:42:13 AM org.apache.pdfb